In [3]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [4]:
# 初始三层神经网络模型

# 定义数据集类
class CustomDataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'rb') as f:
            self.data = pickle.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features, label = self.data[idx]
        features = torch.FloatTensor(features)
        label = torch.tensor(label, dtype=torch.float32)
        return features, label

# 定义神经网络模型
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# 定义训练函数
def train_model(model, train_loader, test_loader, num_epochs=20, learning_rate=0.001, save_path='best_model.pth'):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # 在测试集上评估模型性能
        test_accuracy = evaluate_model(model, test_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Test Accuracy: {test_accuracy}")

        # 如果测试准确率优于当前最佳准确率，则保存模型
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            torch.save(model.state_dict(), save_path)
            print("Model saved!")

# 在测试集上评估模型性能
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    accuracy = correct / total
    return accuracy

# 定义数据加载路径
train_data_path = "./samples/window_2s_train.pkl"
test_data_path = "./samples/window_2s_test.pkl"

# 加载数据集
train_dataset = CustomDataset(train_data_path)
test_dataset = CustomDataset(test_data_path)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 初始化模型
model = Model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 训练模型
train_model(model, train_loader, test_loader)

# 加载最佳模型并在测试集上评估性能
best_model = Model()
best_model.to(device)
best_model.load_state_dict(torch.load('best_model.pth'))
test_accuracy = evaluate_model(best_model, test_loader)
print(f"Final Test Accuracy: {test_accuracy}")


Epoch 1/20, Train Loss: 0.5663429196769382, Test Accuracy: 0.8382406777888186
Model saved!
Epoch 2/20, Train Loss: 0.5505945344854895, Test Accuracy: 0.8382406777888186
Epoch 3/20, Train Loss: 0.5416023432359742, Test Accuracy: 0.8386067674284817
Model saved!
Epoch 4/20, Train Loss: 0.5358445219531885, Test Accuracy: 0.8343182887924272
Epoch 5/20, Train Loss: 0.5314186709479067, Test Accuracy: 0.7608911667799801
Epoch 6/20, Train Loss: 0.527668483149142, Test Accuracy: 0.7989906385649286
Epoch 7/20, Train Loss: 0.524632359237279, Test Accuracy: 0.8150724334501334
Epoch 8/20, Train Loss: 0.5226455439727805, Test Accuracy: 0.7543277025260186
Epoch 9/20, Train Loss: 0.5210308710475391, Test Accuracy: 0.7781235291041263
Epoch 10/20, Train Loss: 0.5192210147713529, Test Accuracy: 0.7999843104440144
Epoch 11/20, Train Loss: 0.517171609973985, Test Accuracy: 0.7621724805188013
Epoch 12/20, Train Loss: 0.5157784610816964, Test Accuracy: 0.7367553998221851
Epoch 13/20, Train Loss: 0.51453009830